# Live/Design: control modes

With *pyAML* it is possible to use the same commands either:
1. live: on the real accelerator
2. design: in simulations
3. errors: in simulations with errors (and for several seeds)
4. live (with a different prefix/host): in a virtual control system (same as in point 1. for those laboratories that have a simulated control system)

Those are called "control modes" and may be used 
- at a global script level (for testing in simulations and then using in real accelerator measurements with a simple switch/comment) 
- anywhere in the script 

Using pyAML on a real accelerator

In [1]:
from pyaml.pyaml import pyaml,PyAML
from pyaml.instrument import Instrument
from pyaml.configuration.factory import Factory
import numpy as np

instantiate a pyAML accelerator

In [2]:
ml:PyAML = pyaml('../../tests/config/EBSTune.yaml')
SR:Instrument = ml.get('sr')

Switch between live and design

In [3]:
# sr = SR.live # use real accelerator
sr = SR.design # run script in simulations

access some magnet families defined in the configuration

In [4]:

quadForTuneDesign = sr.get_magnets("QForTune")
quadForTuneLive = sr.get_magnets("QForTune")


Execute actions with the magnets families (here for example a tune response matrix is evaluated)

In [6]:
# Build tune response matrix
sr.get_lattice().disable_6d()
tune = sr.get_lattice().get_tune()

print(tune)
tunemat = np.zeros((len(quadForTuneDesign),2))

for idx,m in enumerate(quadForTuneDesign):
     str = m.strength.get()
     m.strength.set(str+1e-4)
     dq = sr.get_lattice().get_tune() - tune
     tunemat[idx] = dq*1e4
     m.strength.set(str)

[0.16001787 0.33987387]


At any point in the code it is always possible to still access any of the control modes

In [11]:
print(sr.get_betatron_tune_monitor('BETATRON_TUNE').tune.get())  # what ever the globally set mode whas

# SR.design.get_tune()  # tune from design simulations 

# SR.live.get_tune() # tune from real accelerator

[0.16001787 0.33987387]
